In [1]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from lxml import etree
import requests

In [3]:
headers = {
    'x-devtools-emulate-network-conditions-client-id': "c60c3ba6-26e8-45ce-9f9f-0b5521437cf6",
    'upgrade-insecure-requests': "1",
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
    'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    'referer': "http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247",
    'accept-encoding': "gzip, deflate, sdch",
    'accept-language': "es-ES,es;q=0.8",
    'cookie': "LANGUE_LOGIN=fr",
    'cache-control': "no-cache"
}

data = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247", headers=headers)

soup = BeautifulSoup(data.content, 'html.parser')
queries = soup.find_all('select')
queries_map = {}
for s in queries:
    queries_map[s.get('name')] = {}
    map = queries_map[s.get('name')]
    options = s.find_all('option')
    for o in options:
        if not o.string:
            value = ""
        else:
            value = o.string
        map[value] = o.get('value')

print(queries_map)

{'ww_x_UNITE_ACAD': {'': 'null', 'Cours de mathématiques spéciales': '943282', 'Sciences et technologies du vivant': '945901', 'Science et génie des matériaux': '944917', 'Mathématiques': '944590', 'EME (EPFL Middle East)': '637841336', 'Ingénierie financière': '120623110', 'Architecture': '942293', 'Humanités digitales': '2054839157', 'Génie électrique et électronique ': '943936', 'Génie mécanique': '944263', "Sciences et ingénierie de l'environnement": '942953', 'Chimie et génie chimique': '246696', 'Génie civil': '942623', 'Systèmes de communication': '946228', 'Microtechnique': '945244', 'Physique': '945571', 'Management de la technologie': '946882', 'Section FCUE': '1574548993', 'Informatique': '249847'}, 'ww_x_PERIODE_ACAD': {'': 'null', '2016-2017': '355925344', '2012-2013': '123456101', '2015-2016': '213638028', '2007-2008': '978181', '2014-2015': '213637922', '2008-2009': '978187', '2013-2014': '213637754', '2011-2012': '123455150', '2010-2011': '39486325', '2009-2010': '97819

In [4]:
def getData(unite_acad, periode_acad, periode_pedago, semester_type):
    unite_acad_id = queries_map['ww_x_UNITE_ACAD'][unite_acad]
    periode_acad_id = queries_map['ww_x_PERIODE_ACAD'][periode_acad]
    periode_pedago_id = queries_map['ww_x_PERIODE_PEDAGO'][periode_pedago]
    semester_type_id = queries_map['ww_x_HIVERETE'][semester_type]
    url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter"

    querystring = {"ww_b_list":"1",
                   "ww_i_reportmodel":"133685247",
                   "ww_c_langue":"",
                   "ww_i_reportModelXsl":"133685270",
                   "zz_x_UNITE_ACAD": unite_acad,
                   "ww_x_UNITE_ACAD": unite_acad_id,
                   "zz_x_PERIODE_ACAD": periode_acad,
                   "ww_x_PERIODE_ACAD":periode_acad_id,
                   "zz_x_PERIODE_PEDAGO":periode_pedago,
                   "ww_x_PERIODE_PEDAGO":periode_pedago_id,
                   "zz_x_HIVERETE":semester_type,
                   "ww_x_HIVERETE":semester_type_id,
                   "dummy":"ok"}

    headers = {
        'upgrade-insecure-requests': "1",
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
        'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        'referer': "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247",
        'accept-encoding': "gzip, deflate, sdch",
        'accept-language': "es-ES,es;q=0.8",
        'cookie': "LANGUE_LOGIN=fr",
        'cache-control': "no-cache",
        'postman-token': "65d96955-59bb-daa2-5136-807135f5b89a"
        }

    text = BeautifulSoup(requests.request("GET", url, headers=headers, params=querystring).text, 'html.parser')
    gps = list(text.find_all('table'))[2].find_all('tr')[1].td.a.get('onclick').split('ww_x_GPS=')[1].split("')")[0]
    payload = {'ww_x_GPS': int(gps),
               'ww_i_reportModel':133685247, 
               'ww_i_reportModelXsl': 133685271,
               'ww_x_UNITE_ACAD': unite_acad_id,
               'ww_x_PERIODE_ACAD': periode_acad_id,
               'ww_x_PERIODE_PEDAGO': periode_pedago_id,
               'ww_x_HIVERETE': semester_type_id
              }
    r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.XLS", params=payload)
    return r.text

In [72]:
df_final = None
df_year = None

def addData(unite_acad, periode_acad, periode_pedago, semester_type, list_semesters):
    unite_acad_id = queries_map['ww_x_UNITE_ACAD'][unite_acad]
    periode_acad_id = queries_map['ww_x_PERIODE_ACAD'][periode_acad]
    periode_pedago_id = queries_map['ww_x_PERIODE_PEDAGO'][periode_pedago]
    semester_type_id = queries_map['ww_x_HIVERETE'][semester_type]

    
    soup = BeautifulSoup(getData(unite_acad,periode_acad,periode_pedago,semester_type), 'html.parser')
    rows = soup.find_all('tr')
    header_cols = rows[2].find_all('th')
    col_titles, data = [], {}
    
    for c in header_cols:
        data[c.string] = []
        col_titles.append(c.string)
    [col_titles.append(semestre[9:]) for semestre in list_semesters]
    for semestre in list_semesters:
        data[semestre[9:]] = [ False ] * ( len(rows) - 3)
    col_titles.append('enter')
    data['enter'] = [float("inf")] * ( len(rows) - 3)
    col_titles.append('exit')
    data['exit'] = [float("-inf")] * ( len(rows) - 3)
    
    df = pd.DataFrame(columns=col_titles)
    global df_final, df_year
    if df_final is None:
        df_final = df.copy()
    if df_year is None:
        df_year = df.copy()
    
    
    for h in range(3, len(rows)):
        values = rows[h].find_all('td')
        for v in range(len(values)-2):
            data[header_cols[v].string].append(values[v].string)
        data[periode_pedago[9:]][h-3] =  True 
    df = pd.DataFrame.from_dict(data)
    return df



In [73]:
from IPython.display import display, HTML
def combineData(new_data, old_data, semestre, year):
    for user in new_data['No Sciper'].tolist():
        new_data[['semestre 1', 'semestre 2', 'semestre 3', 'semestre 4', 'semestre 5', 'semestre 6', ]]
        

In [74]:
unite_acad = "Informatique"
periode_acad = list(queries_map['ww_x_PERIODE_ACAD'].keys())
periode_acad.remove('')
periode_pedago = []
for p in list(queries_map['ww_x_PERIODE_PEDAGO'].keys()):
    if 'Bachelor' in p and "b" not in p:
        periode_pedago.append(p)
periode_pedago = sorted(periode_pedago)
df_final = None
#for year in sorted(periode_acad):
    #print(year,)
    #df_year = None
for i in range(1):#periode_pedago[1]:
    a = addData("Informatique", year, periode_pedago[i], "", periode_pedago)
    df_year = combineData(a, df_year, i, year)
display(df_year)
#df_final = df_final.combine_first(df_year)
df_final

   semestre 1 semestre 2
0        True      False
1        True      False
2        True      False
3        True      False
4        True      False
5        True      False
6        True      False
7        True      False
8        True      False
9        True      False
10       True      False
11       True      False
12       True      False
13       True      False
14       True      False
15       True      False
16       True      False
17       True      False
18       True      False
19       True      False
20       True      False
21       True      False
22       True      False
23       True      False
24       True      False
25       True      False
26       True      False
27       True      False
28       True      False
29       True      False
..        ...        ...
60       True      False
61       True      False
62       True      False
63       True      False
64       True      False
65       True      False
66       True      False
67       True      False


None

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filičre opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,semestre 1,semestre 2,semestre 3,semestre 4,semestre 5,semestre 6,enter,exit


In [8]:
df_final['No Sciper'].tolist()

['169569',
 '174905',
 '173922',
 '179406',
 '179428',
 '179324',
 '174597',
 '179449',
 '178553',
 '179426',
 '178271',
 '182433',
 '180731',
 '171619',
 '179837',
 '179157',
 '179864',
 '174590',
 '178843',
 '178711',
 '178786',
 '179567',
 '176282',
 '178656',
 '181445',
 '178718',
 '175466',
 '173882',
 '181612',
 '181232',
 '178706',
 '180284',
 '181121',
 '170509',
 '175379',
 '180570',
 '178604',
 '175190',
 '178660',
 '181248',
 '179163',
 '181181',
 '181244',
 '175685',
 '169731',
 '175001',
 '181424',
 '181259',
 '178433',
 '181460',
 '181298',
 '181513',
 '175478',
 '176459',
 '175014',
 '181514',
 '179355',
 '181076',
 '175576',
 '181115',
 '180094',
 '180853',
 '178726',
 '181017',
 '175031',
 '179194',
 '175754',
 '179980',
 '179988',
 '174187',
 '180959',
 '171195',
 '178620',
 '180979',
 '180980',
 '178948',
 '169795',
 '178684',
 '180241',
 '180982',
 '181291',
 '175280',
 '179053',
 '180854',
 '171568',
 '174120',
 '180185',
 '175834',
 '174340',
 '178682',
 '169569',